El primer paso es instalar las dependencia necesarias para iniciar el proyecto

In [ ]:
%pip install langchain

In [ ]:
%pip install -qU langchain-openai

A continuación se establece una variable de entorno con la clave de API para conectarse a OpenAI, además especificamos que se utilizará el modelo gpt-4 de chatgpt

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

Para probar su funcionamiento se crea una lista llamada messages que contiene dos mensajes:

SystemMessage: Da instrucciones al modelo para traducir lo que sigue de inglés a italiano.
HumanMessage: Contiene el mensaje "hi!", que se desea traducir.

In [16]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e3e94e24-b1d3-4edb-a432-8491627b4d7a-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

En este paso se toma la salida del modelo y la convierte a una cadena de texto de acuerdo con la configuración de StrOutputParser

In [17]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke(messages)

parser.invoke(result)

'Ciao!'

A continuación se usa el operador | para combinar el modelo y el parser. Esto forma una cadena (chain) en la que model procesa los mensajes primero y luego pasa el resultado automáticamente al parser para su formateo

In [18]:
chain = model | parser

chain.invoke(messages)

'ciao!'

En este paso se crea una instancia de ChatPromptTemplate llamada prompt_template usando from_messages. Este método toma una lista de mensajes, donde cada elemento tiene un tipo de mensaje ("system" o "user") y una cadena que contiene el contenido del mensaje.

El mensaje system utiliza system_template, que indica al modelo que traduzca un texto a un idioma específico.
El mensaje user usa "{text}", que es otro marcador de posición que se rellenará con el contenido que el usuario desea traducir (en este caso, "hi")

In [19]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})])

Con la siguiente linea se convierte el resultado obtenido a un formato de lista de objetos, esta lista tendrá cada mensaje en el formato que necesita el modelo para entender la estructura de la conversación

In [20]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]

Finalmente se usa el operador | para combinar prompt_template, model, y parser en una cadena llamada chain. Esto permite que prompt_template procese los datos de entrada y genere un prompt formateado, este prompt se apsa automáticamente a model, el cuál genera una respuesta para que la salida de model se pase finalmente a parser, que formatea o procesa la respuesta.

In [21]:
chain = prompt_template | model | parser

chain.invoke({"language": "italian", "text": "hi"})

'ciao'